In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

In [15]:
ord('A') - 65

0

In [19]:
def parse_minutes(x):
    
    spl = x.split('.')[0].split(' ')
    hours = int(spl[0]) * 60
    mins = int(spl[2]) if spl[2] != '' else 0
    return hours + mins

def read_map(plate_map, ctrl='H', num_inducers=1, induction_lvl=6, ctrl_replicate=2):
    
    plate_map = plate_map.melt(id_vars=['group'])
    plate_map['variable'] = plate_map['variable'].astype(int)
    plate_map['Well'] = plate_map['group'] + plate_map['variable'].apply(lambda x: '{:02d}'.format(x))
    plate_map.loc[~plate_map['group'].isin([ctrl]), 'suffix'] = (plate_map['group'].apply(lambda x: ord(x)) - 65).astype(str) + ((plate_map['variable'] - 1) % induction_lvl).astype(str)
    plate_map.loc[plate_map['group'].isin([ctrl]), 'suffix'] = (plate_map['variable'] - 1) % ctrl_replicate
    return plate_map[['Well', 'value', 'suffix']].dropna()

def generate_metadata(well, plate_map):
    
    dictionary = pd.read_csv('datasets/dictionary.csv')
    sample_map = {
        'P62': 'pBAD-rJ/Ara',
        'P63': 'araC-pBAD-rJ/Ara',
        'P64': 'pCin-rJ/OHC14',
        'P65': 'pCymRC-rJ/Cuma',
        'P66': 'cymRC-pCymRC-rJ/Cuma',
        'P67': 'pLux2-rJ/AHL',
        'P68': 'pLuxB-rJ/AHL',
        'P69': 'pPhlF-rJ/DAPG',
        'P70': 'pSalTTC-rJ/Sal',
        'P71': 'pVanCC-rJ/Van',
        'P72': 'rhaS-pRhaB-rJ/Rha',
        'P73': 'araC-pBAD/Ara',
        'E720': 'e11x33',
        'E721': 'e15x33',
        'E722': 'e16x33',
        'E723': 'e20x33',
        'E724': 'e32x33',
        'E725': 'e34x33',
        'E726': 'e38x33',
        'E727': 'e41x33',
        'E728': 'e42x33',
        'G726': 'e11x32STPhoRadA',
        'G721': 'e15x32NpuSspS2',
        'G722': 'e16x33NrdA2',
        'G723': 'e20x32gp411',
        'G724': 'e32x30SspGyrB',
        'G725': 'e34x30MjaKlbA',
        'G726': 'e38x32gp418',
        'G727': 'e41x32NrdJ1',
        'G728': 'e42x32STIMPDH1',
        'A65': 'e16x30SspGyrB',
        'A191': 'e32x33NrdJ1',
        'BK': 'blank-kan',
        'BA': 'blank-amp',
        '3K3-N': 'negative-control-kan',
        '4A3-N': 'negative-control-amp',
        '4A3-P': 'positive-control-amp'
    }
    #sample_map.update(control_map)
    df = pd.merge(well, plate_map, on='Well', how='left').dropna(subset=['value']).reset_index(drop=True)
    df['short_name'] = df['value'].map(sample_map) + '_' + df['suffix'].astype(int).astype(str)
    return df.dropna()


def transpose_data(df, col):
    
    df.set_index(col, inplace=True)
    df = df.transpose().reset_index()
    df['time'] = df['index'].apply(parse_minutes)
    df = df.set_index('time').drop('index', axis=1)
    return df

def generate_data(df, name, h=24, m=0, num_data=4, start_idx=3, col='short_name'):
    
    datas = []
    idxs = [start_idx]
    for i in range(1, num_data+1):
        idxs.append(h * 3 + idxs[i-1] + 1 + int(m/20))
        data = (df.iloc[:, idxs[i-1]:idxs[i]]).astype(float)
        data = pd.concat([name, data], axis=1)
        data = transpose_data(data, col)
        datas.append(data)
    return datas

In [20]:
folder = '028-A65-A191-repeat'
plate_map = read_map(pd.read_csv('datasets/experiment/{}/plate_map.csv'.format(folder)))
raw_data = pd.read_csv('datasets/experiment/{}/raw.csv'.format(folder), skiprows=[0])
metadata = generate_metadata(raw_data['Well'], plate_map)
data = raw_data[raw_data['Well'].isin(metadata['Well'])].reset_index(drop=True)
datas = generate_data(data, metadata['short_name'], h=23, m=0, num_data=4)

In [21]:
datas[0]

short_name,e16x30SspGyrB_0,e16x30SspGyrB_1,e16x30SspGyrB_2,e16x30SspGyrB_3,e16x30SspGyrB_4,e16x30SspGyrB_5,e32x33NrdJ1_0,e32x33NrdJ1_1,e32x33NrdJ1_2,e32x33NrdJ1_3,e32x33NrdJ1_4,e32x33NrdJ1_5,e16x30SspGyrB_0,e16x30SspGyrB_1,e16x30SspGyrB_2,e16x30SspGyrB_3,e16x30SspGyrB_4,e16x30SspGyrB_5,e32x33NrdJ1_0,e32x33NrdJ1_1,e32x33NrdJ1_2,e32x33NrdJ1_3,e32x33NrdJ1_4,e32x33NrdJ1_5,e16x30SspGyrB_0,e16x30SspGyrB_1,e16x30SspGyrB_2,e16x30SspGyrB_3,e16x30SspGyrB_4,e16x30SspGyrB_5,e32x33NrdJ1_0,e32x33NrdJ1_1,e32x33NrdJ1_2,e32x33NrdJ1_3,e32x33NrdJ1_4,e32x33NrdJ1_5,e16x30SspGyrB_0,e16x30SspGyrB_1,e16x30SspGyrB_2,e16x30SspGyrB_3,e16x30SspGyrB_4,e16x30SspGyrB_5,e32x33NrdJ1_0,e32x33NrdJ1_1,e32x33NrdJ1_2,e32x33NrdJ1_3,e32x33NrdJ1_4,e32x33NrdJ1_5,e16x30SspGyrB_0,e16x30SspGyrB_1,e16x30SspGyrB_2,e16x30SspGyrB_3,e16x30SspGyrB_4,e16x30SspGyrB_5,e32x33NrdJ1_0,e32x33NrdJ1_1,e32x33NrdJ1_2,e32x33NrdJ1_3,e32x33NrdJ1_4,e32x33NrdJ1_5,e16x30SspGyrB_0,e16x30SspGyrB_1,e16x30SspGyrB_2,e16x30SspGyrB_3,e16x30SspGyrB_4,e16x30SspGyrB_5,e32x33NrdJ1_0,e32x33NrdJ1_1,e32x33NrdJ1_2,e32x33NrdJ1_3,e32x33NrdJ1_4,e32x33NrdJ1_5,blank-kan_0,blank-kan_1,negative-control-kan_0,negative-control-kan_1,blank-amp_0,blank-amp_1,negative-control-amp_0,negative-control-amp_1,positive-control-amp_0,positive-control-amp_1
time,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,0.008,0.025,0.024,0.030,0.003,0.020,0.037,0.058,0.032,0.014,0.031,0.063,0.029,0.00005,-0.002,0.155,0.014,-0.002,0.001,0.034,0.031,-0.006,0.040,0.043,0.023,-0.002,0.003,0.006,0.054,-0.00095,0.018,0.048,0.062,0.008,0.058,0.020,0.038,0.00075,-0.005,0.002,-0.00025,0.00025,0.006,0.016,0.055,0.007,0.004,0.039,0.071,-0.002,0.007,0.046,0.013,0.010,0.004,0.030,0.049,-0.00065,0.001,0.063,0.044,0.011,0.027,0.021,0.015,0.033,0.029,0.023,0.044,0.032,0.017,0.132,NaN,NaN,0.264,0.061,NaN,NaN,-0.00085,-0.00005,0.159,1.005
20,0.023,0.025,0.017,0.018,0.018,0.019,0.019,0.081,0.018,0.015,0.023,0.032,0.016,0.01700,0.008,0.013,0.013,0.011,0.012,0.014,0.012,0.013,0.023,0.028,0.016,0.013,0.014,0.016,0.049,0.01200,0.015,0.015,0.021,0.017,0.016,0.023,0.015,0.01400,0.011,0.014,0.01400,0.01300,0.015,0.016,0.018,0.015,0.013,0.016,0.015,0.013,0.014,0.012,0.019,0.015,0.016,0.016,0.016,0.01400,0.015,0.025,0.018,0.017,0.017,0.017,0.017,0.019,0.017,0.017,0.016,0.017,0.018,0.024,NaN,NaN,0.044,0.051,NaN,NaN,0.02600,0.02400,0.042,0.184
40,0.023,0.027,0.019,0.019,0.019,0.020,0.020,0.080,0.019,0.017,0.025,0.030,0.019,0.01900,0.011,0.014,0.016,0.014,0.013,0.014,0.012,0.014,0.023,0.026,0.019,0.014,0.014,0.017,0.046,0.01300,0.015,0.015,0.022,0.019,0.017,0.023,0.016,0.01500,0.015,0.015,0.01400,0.01300,0.016,0.018,0.276,0.017,0.014,0.017,0.016,0.016,0.014,0.015,0.020,0.015,0.017,0.018,0.016,0.01500,0.015,0.027,0.021,0.019,0.018,0.019,0.020,0.020,0.019,0.019,0.017,0.018,0.035,0.025,NaN,NaN,0.048,0.051,NaN,NaN,0.02100,0.02600,0.046,0.189
60,0.030,0.034,0.025,0.026,0.023,0.025,0.024,0.075,0.024,0.022,0.029,0.036,0.021,0.02300,0.013,0.016,0.019,0.017,0.016,0.020,0.017,0.017,0.027,0.031,0.021,0.020,0.017,0.019,0.045,0.02000,0.021,0.020,0.024,0.022,0.021,0.027,0.021,0.02000,0.019,0.019,0.01900,0.05800,0.022,0.021,0.269,0.024,0.029,0.024,0.021,0.019,0.019,0.019,0.022,0.052,0.021,0.022,0.022,0.02100,0.020,0.033,0.024,0.023,0.025,0.023,0.023,0.025,0.025,0.024,0.024,0.025,0.080,0.030,NaN,NaN,0.053,0.058,NaN,NaN,0.02400,0.03000,0.051,0.215
80,0.043,0.046,0.037,0.037,0.037,0.038,0.035,0.095,0.033,0.032,0.038,0.046,0.032,0.03200,0.021,0.026,0.028,0.026,0.025,0.028,0.025,0.027,0.037,0.044,0.031,0.030,0.026,0.029,0.060,0.02900,0.026,0.027,0.034,0.029,0.028,0.036,0.029,0.06700,0.028,0.029,0.02900,0.05200,0.032,0.032,0.287,0.031,0.028,0.033,0.028,0.059,0.080,0.027,0.029,0.059,0.032,0.034,0.031,0.03000,0.053,0.042,0.035,0.034,0.033,0.034,0.032,0.035,0.034,0.035,0.034,0.035,0.068,0.039,NaN,NaN,0.067,0.069,NaN,NaN,0.03000,0.03700,0.059,0.257
100,0.069,0.071,0.060,0.059,0.059,0.060,0.052,0.116,0.054,0.051,0.059,0.066,0.048,0.04800,0.033,0.040,0.043,0.0

In [11]:
datas[0].to_csv('datasets/experiment/{}/ods.csv'.format(folder))
datas[1].to_csv('datasets/experiment/{}/fluos.csv'.format(folder))
datas[2].to_csv('datasets/experiment/{}/fluos-lower-1.csv'.format(folder))
datas[3].to_csv('datasets/experiment/{}/fluos-lower-2.csv'.format(folder))
#datas[4].to_csv('datasets/experiment/{}/raw-ods.csv'.format(folder))
#datas[5].to_csv('datasets/experiment/{}/raw-fluos.csv'.format(folder))
#datas[6].to_csv('datasets/experiment/{}/raw-fluos-lower-1.csv'.format(folder))
#datas[7].to_csv('datasets/experiment/{}/raw-fluos-lower-2.csv'.format(folder))